# RSP_Evals
This notebook reviews model runs located in a given RSP repository and pulls metrics from each RSP model run and compiles them into a single CSV. 

Model users should run a Batch file titled 'GetRSPCorridorInfo_SingleProject.bat,' to generate necessary output files. (If these files do not exist when running this script, however, the script will automatically generate them.)

## Inputs/Outputs
The list below outlines the scripts that are used in this notebook, and the outputs that are created from each script.
- **export_geog.py**
    - this script exports link-level highway project data from Emme (or segment-level transit project data), based on Select_Link/Select_Line files
    - **OUTPUTS**:
        - emme_links.shp, located in '..\Database\Select_Link' (for highway projects)
        - emme_tsegs.shp, located in '..\Database\Select_Line' (for transit projects)
- **select_by_location.py**
    - for highway projects, this script selects and exports corridor-level links for both the RSP model run and no-build run.
    - for transit projects, this script exports segment-level data from Emme of transit networks for each transit assignment scenario (721, 723, 725, and 727).
    - **OUTPUTS**:
        - *transitpunch.csv, located in '..\Database\data\' (for transit projects)*
        - *nb_corridor_70029.csv, located in '..\Database\Select_Link\' (for highway projects)*
        - *rsp_corridor_70029.csv, located in '..\Database\Select_Link\' (for highway projects)*
- **congestion_metrics_EDA.py** 
    - this script calculates congestion-related measures like vehicle hours traveled
    - **OUTPUTS**:
        - *RSP_congestion_factors.csv, located in '..\Database\rsp_evaluation\results\'*
- **rsp_emissions_2.py**
    - This script calculates estimated greenhouse gas emissions based on link volumes and speeds
    - **OUTPUTS**:
        - *emissions.csv, located in '..\Database\rsp_evaluation\results\'*

### 1. Import Packages and Point to Inputs

In [58]:
##############################################################################
# BCA Calculations - Emme Punch, PMs, and BCA Calcs
##############################################################################
# Tim O'Leary - 2023/08/28
# ------------------------------
# The following notebook reviews model runs located in the RSP repository, 
# checks for relevant model outputs (and, if they don't exist, recreate them), 
# then does light calculations to derive metrics for RSP evaluations. 
#
# This script uses outputs from congestion_metrics_EDA.py and 
#
##############################################################################

## IMPORT PACKAGES
## ------------------
print('BEGIN RSP EVALUATIONS.')
import pandas as pd
import subprocess
import os
import fnmatch
import csv
import datetime as dt
print('imported packages')

##------------------------------------
## INPUT FILES AND PARAMETERS
##------------------------------------

#output csv
timestamp = dt.datetime.now()
timestamp = timestamp.strftime("%Y%m%d")
final_output = os.path.join(os.getcwd(),f'RSP_Comparison_{timestamp}.csv')
details = os.path.join(os.getcwd(),'details.txt')

BEGIN RSP EVALUATIONS.
imported packages


In [59]:
#folder containing RSP runs
dir = 'E:/tko/TPAT/RSP_Evals/Test_repo'
src = os.path.join(os.getcwd(), 'src')
print('RSP repository location: \n', dir, '\n')

#create dictionary of rsp runs
runs = fnmatch.filter(os.listdir(dir), '*RSP*')
nobuilds = [run for run in runs if 'RSP00' in run]
nobuild = nobuilds[0]
rsp_runs = [run for run in runs if run not in nobuilds]

runs_dir = [dir+r'\{}'.format(d)+r'\cmap_trip-based_model' for d in runs]
rsps_dir = dict([[d, os.path.join(dir,f'{d}','cmap_trip-based_model')] for d in rsp_runs])   ##-- dictionary of runs of format {'RSP_20': <full filepath>} 
nobuild_dir = [os.path.join(dir,f'{d}','cmap_trip-based_model') for d in nobuilds][0]


#check to make sure nobuild and rsp_runs exist and are non-empty
if len(nobuilds) != 1 or len(runs) == 0:
    raise ValueError(f'''Somethin' ain't right. Hold yer horses an' check the following, partner: 
        - Is this the correct folder of RSP model runs?: {dir} 
        - Is the base year model run labeled using 'RSP00' in the name? 
        - Is there more than one base year model run? If so, remove all but one.
    ''')

print("    no-build run:", nobuild)
print("    list of model runs:", rsp_runs)

#check whether rsp is highway or transit, 
#then create dictionary of rsp info: 
#{<rsp_name>: [<model run filepath>, <'link' or 'line'>, <select_link or select_line file>]}
rsp_runs_dir = {}

for run in rsps_dir:
    rspyr = run.split('_')
    rsp = rspyr[0]

    #first determine whether run is highway or transit
    slink_file = os.path.join(rsps_dir[run], f'Database/Select_Link/{rsp}_links.txt')
    sline_file = os.path.join(rsps_dir[run], f'Database/Select_Line/{rsp}_line.txt')
    link = os.path.exists(slink_file) # truthy, looking for select_links.txt
    line = os.path.exists(sline_file) # truthy, looking for select_line.txt
    if link and line:
        raise ValueError(f'There are both select_line AND select_link files in {run} when only one is permitted for RSP evaluations.')
    if not line and not link:
        raise ValueError(f'No select link or select line file detected in {run}. Check if ../Database/Select_Link/{rsp}_links.txt exists (or {rsp}_line.txt).')
    if link and not line:   #roadway projects
        rsp_runs_dir[run] = [rsps_dir[run], 'link', slink_file]
    elif line and not link: #transit projects
        rsp_runs_dir[run] = [rsps_dir[run], 'line', sline_file]
    else:
        raise ValueError(f"Something isn't right. {run} has more than one Select_Link or Select_Line file when only one (or none) is permitted.")

##------------------

#grabbing parameters csv that contains various assumptions for calculations
#bca_parameters.csv should be located in "..\Test_repo\Evals," same folder as this notebook
bcaparams_dir = os.path.join(src,'bca_parameters.csv')

#bring in parameters file as a dictionary (easier to call)
params={}
with open(bcaparams_dir, 'r') as file:
    csvreader = csv.reader(file)
    next(csvreader) #skip first row of headers
    for row in csvreader:
        params[row[0]] = float(row[1]) #first entry (row[0]) is parameter name, second (row[1]) is value

##------------------

RSP repository location: 
 E:/tko/TPAT/RSP_Evals/Test_repo 

    no-build run: RSP00_700
    list of model runs: ['RSP20_700', 'RSP57_700']


### 2. Gather RSP Info, and Data Check

In [60]:
## DATA CHECK
##------------------

print('Checking for necessary data...')

# create necessary data -- only done if data wasn't previously created

# (1) for highway projects, we need:
#       - corridor-level link information (on both rsp network and nobuild network)
#       - RSP_congestion_factors.csv (output of congestion_metrics_EDA.py)

# (2) for transit projects, we need transitpunch.csv, which is an export of all transit segments for each time of day

# (3) for nobuild, we need emme_links.shp (export of network) and transitpunch.csv

##########################################
## CHECK NOBUILD RUN FOR NECESSARY DATA ##
##########################################

print(f'Checking {nobuild} (nobuild)...')

#corridor links
file = os.path.join(nobuild_dir,'Database\\Select_Link\\scen_70029\\emme_links.shp')
if not os.path.exists(file):
    print(f'{nobuild} is missing geographic data. Creating data now...')
    subprocess.run([
        os.path.join(src,'GetRSPCorridorInfo_SingleProject.bat'), 
        nobuild_dir, 
        nobuild_dir
    ])
    print(f'{nobuild} geographic data complete')
else:
    print(f'{nobuild} geographic data exists')

#(2) RSP_congestion_factors.csv
file = os.path.join(nobuild_dir,'Database/rsp_evaluation/results/RSP_congestion_factors.csv')
if not os.path.exists(file):
    print(f'{nobuild} is missing congestion-related data. Creating data now...')
    subprocess.run(
        ["python", os.path.join(src,'congestion_metrics_EDA.py'), nobuild_dir, nobuild, bcaparams_dir],
        capture_output=True,
        text=True
    )
    print(f'{nobuild} congestion-related data complete')
else:
    print(f'{nobuild} congestion-related data exists')

#(3) emissions.csv
file = os.path.join(nobuild_dir,'Database/rsp_evaluation/results/emissions.csv')
if not os.path.exists(file):
    print(f'{nobuild} is missing emissions-related data. Creating data now...')
    subprocess.run(
        ["python", os.path.join(src,'rsp_emissions_2.py'), nobuild_dir],
        capture_output=True,
        text=True
    )
    print(f'{nobuild} emissions-related data complete')
else:
    print(f'{nobuild} emissions-related data exists')

#(4) transitpunch.csv
file = os.path.join(nobuild_dir,'Database/data/transitpunch.csv')
if not os.path.exists(file):
    print(f'{nobuild} is missing transit link data. Creating data now...')
    run_folder = os.path.dirname(nobuild_dir)
    subprocess.run([os.path.join(src, 'GetRSPCorridorInfo_SingleProject.bat'), run_folder, run_folder])    
    print(f'{nobuild} transit link data complete')
else:
    print(f'{nobuild} transit link data exists')

###################################
## CHECK RSPs FOR NECESSARY DATA ##
###################################

#in the following, rsp_runs_dir changes its format from 
# {<rsp_name>: <rsp_file_location>}
# -to-
# {<rsp_name>: [<rsp_filepath>, <rsp_type>, <rsp_project_link_filepath>]}
#
# this dictionary will be used to call on various things
for run in rsp_runs_dir:

    #(1) emissions.csv
    file = rsp_runs_dir[run][0]+f'\\Database\\rsp_evaluation\\results\\emissions.csv'
    if rsp_runs_dir[run][1] == 'link' and not os.path.exists(file):
        print(f'{run} is missing emissions-related data. Creating data now...')
        subprocess.run(
            ["python", os.getcwd()+'\\rsp_emissions_2.py', rsp_runs_dir[run][0]]
        )
    else:
        print(f'{run} emissions-related data found')

    #HIGHWAY PROJECTS
    # checking for necessary data by rsp type
    # rsp_runs_dir = {<rsp_name>: [<rsp_filepath>, <rsp_type>, <rsp_project_link_filepath>]}

    if rsp_runs_dir[run][1] == 'link':
        #see if corridor links exist
        dir = [rsp_runs_dir[run][0]+f'\\Database\\Select_Link\\{a}_corridor_70029.csv' for a in ['rsp', 'nb']]
        if os.path.exists(dir[0]) and os.path.exists(dir[1]):
            print(f'{run} geographic data found')
            continue
        else:
            print(f'{run} is missing geographic data. Creating data now...')
            #if don't exist, run GetRSPCorridorInfo_SingleProject.bat
            subprocess.run([os.getcwd()+'\\GetRSPCorridorInfo_SingleProject.bat', nobuild_dir, rsp_runs_dir[run][0]])

        #see if RSP_congestion_factors.csv exists
        file = rsp_runs_dir[run][0]+f'\\Database\\rsp_evaluation\\results\\RSP_congestion_factors.csv'
        if not os.path.exists(file):
            print(f'{run} is missing congestion-related data. Creating data now...')
            #if don't exist, run congestion_metrics_EDA.py
            subprocess.run(
                ["python", os.getcwd()+'\\congestion_metrics_EDA.py', rsp_runs_dir[run][0], run, bcaparams_dir],
                capture_output=True,
                text=True
            )
        else:
            print(f'{run} congestion-related data found')

    #TRANSIT PROJECTS
    if rsp_runs_dir[run][1] == 'line':
        file = rsp_runs_dir[run][0]+f'\\Database\\data\\transitpunch.csv'
        if not os.path.exists(file):
            print(f'{run} is missing transit link data. Creating data now...')
            #if don't exist, run GetRSPCorridorInfo_SingleProject.bat
            subprocess.run([os.getcwd()+'\\GetRSPCorridorInfo_SingleProject.bat', nobuild_dir, rsp_runs_dir[run][0]])
        else:
            print(f'{run} transit link data found')


print('Data check completed!')

Checking for necessary data...
Checking RSP00_700 (nobuild)...
RSP00_700 geographic data exists
RSP00_700 congestion-related data exists
RSP00_700 emissions-related data exists
RSP00_700 transit link data exists
RSP20_700 emissions-related data found
RSP20_700 geographic data found
RSP57_700 emissions-related data found
RSP57_700 transit link data found
Data check completed!


### 3. Performance Measures Calculations

In [61]:
# #create csv columns -- NO LONGER USED

# #column names

# columns = [
#     'ID', 
#     'measure_pavement_age',
#     'measure_pavement_condition',
#     'measure_safety',
#     'measure_mobility',
#     'measure_reliability',
#     'measure_change_in_vmt',
#     'measure_change_in_congested_vht_in_corridor',
#     'measure_change_in_regional_work_trip_travel_time',
#     'measure_change_in_work_trip_travel_time_in_corridor',
#     'measure_change_in_job_accessibility',
#     'measure_change_in_fatalities_and_serious_injuries_per_year',
#     'measure_change_in_congested_vht_for_heavy_trucks_in_corridor',
#     'measure_freight_improvement',
#     'measure_change_in_co2_emissions',
#     'measure_change_in_pm_emissions',
#     'measure_change_in_development_pressure_in_conservation_areas',
#     'measure_direct_impact_on_conservation_areas',
#     'measure_change_in_impervious_area',
#     'measure_project_use_by_residents_of_economically_disconnected_areas',
#     'measure_change_in_fine_particulate_matter_emissions_in_economically_disconnected_areas',
#     'measure_change_in_access_to_1_barrier_jobs_for_economically_disconnected_areas',
#     'measure_economic_impact_due_to_industry_clustering',
#     'measure_benefit_to_key_industries',
#     'measure_benefits_to_areas_with_industrial_vacancy',
#     'measure_transit_asset_condition',
#     'measure_transit_reliability',
#     'measure_change_in_regional_transit__ridership',
#     'measure_support_of_infill_development',
#     'measure_change_in_congested_vehicle_hours_traveled_in_region',
#     'measure_change_in_congested_vht_for_heavy_trucks_in_region',
#     'measure_bike_ped_stuff'
# ]
# col_val = dict([[x, None] for x in columns])

In [62]:
# GATHER NO-BUILD DATA
nblink = pd.read_csv(nobuild_dir+'\\Database\\rsp_evaluation\\results\\RSP_congestion_factors_links.csv')
nb_trnt = pd.read_csv(nobuild_dir+'\\Database\\data\\transitpunch.csv')

In [64]:
# CALCULATIONS FOR EACH RSP, AND WRITE INTO CSV

rows = []
for run in rsp_runs_dir:
    col_val = {}
    col_val['ID'] = run
    #gather relevant data
    runtype = rsp_runs_dir[run][1]

    if runtype == 'link':
        ## -- gather data
        #gather project link info
        plinks = rsp_runs_dir[run][2]
        plinks = pd.read_csv(plinks, header=None, names=['inode','jnode'], skiprows=1)
        plinks['inode']=plinks['inode'].str[2:]
        plinks[['inode','jnode']]=plinks[['inode','jnode']].astype(int)
        plinks['rsp'] = 1

        #create rsp network, and identify corridor links for both nb and rsp networks
        rsplink = rsp_runs_dir[run][0]+'\\Database\\rsp_evaluation\\results\\RSP_congestion_factors_links.csv'
        rsplink = pd.read_csv(rsplink)

    if runtype == 'line':  
        rsp_trnt = pd.read_csv(rsp_runs_dir[run][0]+'\\Database\\data\\transitpunch.csv')

    print(f'Calculations for {run} ({rsp_runs_dir[run][1]})')

    # 2 - measure_pavement_age
    # 3 - measure_pavement_condition
    # print('--pavement age and condition')
    col_val['measure_pavement_age'] = None
    col_val['measure_pavement_condition'] = None

    # 4 - measure_safety
    # print('--safety')
    col_val['measure_safety'] = None

    # 5 - measure_mobility
    # print('--mobility')
    col_val['measure_mobility'] = None

    # 6 - measure_reliability
    # print('--reliability')
    col_val['measure_reliability'] = None

    # 7 - measure_change_in_vmt
    if runtype == 'link':
        print('--change in regional vmt')
        orig_vmt = nblink['AllVMT'].sum()
        rsp_vmt = rsplink['AllVMT'].sum()
        change_vmt = rsp_vmt - orig_vmt
        print('    ', int(change_vmt), 'VMT')
        
    else:
        change_vmt = None
    col_val['measure_change_in_vmt'] = change_vmt

    # 8 - measure_change_in_congested_vht_in_corridor
    if runtype == 'link':
        rsp_name='RSP20_700'

        nb_corridor = pd.read_csv(rsp_runs_dir[rsp_name][0]+'\\Database\\Select_Link\\nb_corridor_70029.csv')
        nb_corridor = nb_corridor[['INODE','JNODE']].copy()
        nb_corridor.rename(columns={'INODE':'inode','JNODE':'jnode'}, inplace=True)
        nb_corridor['corridor'] = 1
        nb = pd.merge(nblink, nb_corridor, on=['inode','jnode'], how='left')
        nb.loc[nb['corridor'].isnull(),'corridor'] = 0

        rsp_corridor = pd.read_csv(rsp_runs_dir[rsp_name][0]+'\\Database\\Select_Link\\rsp_corridor_70029.csv')
        rsp_corridor = rsp_corridor[['INODE','JNODE']].copy()
        rsp_corridor.rename(columns={'INODE':'inode','JNODE':'jnode'}, inplace=True)
        rsp_corridor['corridor'] = 1
        rsp = pd.merge(rsplink, rsp_corridor, on=['inode','jnode'], how='left')
        rsp.loc[rsp['corridor'].isnull(),'corridor'] = 0

        nb_cvht = nb.groupby('corridor').agg({'CongestedVHT':'sum'})
        rsp_cvht = rsp.groupby('corridor').agg({'CongestedVHT':'sum'})

        c_orig_cvht = nb_cvht.loc[1,'CongestedVHT'] 
        c_rsp_cvht = rsp_cvht.loc[1,'CongestedVHT']
        c_change_cvht = c_rsp_cvht - c_orig_cvht

        print('--change in corridor congested VHT')
        print('    ', int(c_change_cvht), 'VHT')
    else:
        c_change_cvht = None
    col_val['measure_change_in_congested_vht_in_corridor'] = c_change_cvht

    # 9 - measure_change_in_regional_work_trip_travel_time
    # print('--change in regional work trip travel time')
    col_val['measure_change_in_regional_work_trip_travel_time'] = None

    # 10 - measure_change_in_work_trip_travel_time_in_corridor
    # print('--change in corridor work trip travel time')
    col_val['measure_change_in_work_trip_travel_time_in_corridor'] = None

    # 11 - measure_change_in_job_accessibility
    # print('--change in job accessibility')
    col_val['measure_change_in_job_accessibility'] = None

    # 12 - measure_change_in_fatalities_and_serious_injuries_per_year
    if runtype == 'link':
        ## K+A -- calculations are per 100M VMT
        #non-interstate rate
        nblink['annual_ka'] = nblink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nikarate']
        nblink['annual_crash'] = nblink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nicrashrate']
        #interstate rate
        nblink.loc[nblink['vdf'].isin([2,3,4,5,8]), 'annual_ka'] = nblink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_ikarate']
        nblink.loc[nblink['vdf'].isin([2,3,4,5,8]), 'annual_crash'] = nblink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nicrashrate']
        #non-interstate rate
        rsplink['annual_ka'] = rsplink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nikarate']
        rsplink['annual_crash'] = rsplink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nicrashrate']
        #interstate rate
        rsplink.loc[rsplink['vdf'].isin([2,3,4,5,8]), 'annual_ka'] = rsplink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_ikarate']
        rsplink.loc[rsplink['vdf'].isin([2,3,4,5,8]), 'annual_crash'] = rsplink['AllVMT'] / 100000000 * params['ann_factor'] * params['SAFE_nicrashrate']

        orig_ka = nblink['annual_ka'].sum()
        rsp_ka = rsplink['annual_ka'].sum()
        change_ka = rsp_ka - orig_ka
        print('--change in annual roadway fatalities and serious injuries')
        print('    ', int(change_ka), 'fatalities and serious injuries')
    else:
        change_ka = None
    col_val['measure_change_in_fatalities_and_serious_injuries_per_year'] = change_ka

    # 13 - measure_change_in_congested_vht_for_heavy_trucks_in_corridor
    if runtype == 'link':
        orig_htruck_cvht_c = nb.loc[nb['corridor']==1, 'CongestedHTruckVHT'].sum()
        rsp_htruck_cvht_c = rsp.loc[rsp['corridor']==1, 'CongestedHTruckVHT'].sum()
        change_htruck_cvht_c = rsp_htruck_cvht_c - orig_htruck_cvht_c
        print('--change in congested VHT for heavy trucks in corridor')
        print('    ', int(change_htruck_cvht_c), 'VHT')
    else:
        change_htruck_cvht_c = None
    col_val['measure_change_in_congested_vht_for_heavy_trucks_in_corridor'] = change_htruck_cvht_c

    # 14 - measure_freight_improvement
    # print('--freight improvement')
    col_val['measure_freight_improvement'] = None

    # 15 - measure_change_in_greenhouse_gas_emissions
    # -- I split this into two, based on rsp_emissions.py results: co2, and pm
    orig_emissions = pd.read_csv(nobuild_dir+'\\Database\\rsp_evaluation\\results\\emissions.csv')
    orig_co2 = orig_emissions.iloc[0,2]
    orig_pm = orig_emissions.iloc[0,3]
    rsp_emissions = pd.read_csv(rsp_runs_dir[run][0]+'\\Database\\rsp_evaluation\\results\\emissions.csv')
    rsp_co2 = rsp_emissions.iloc[0,2]
    rsp_pm = rsp_emissions.iloc[0,3]
    change_co2 = rsp_co2 - orig_co2
    change_pm = rsp_pm - orig_pm
    print('-- change in ghg emissions')
    print('    CO2: ', change_co2, 'tons')
    print('    PM:  ', change_pm, 'tons')
    col_val['measure_change_in_co2_emissions'] = change_co2
    col_val['measure_change_in_pm_emissions'] = change_pm

    # 16 - measure_change_in_development_pressure_in_conservation_areas
    # print('--development pressure in conservation areas')
    col_val['measure_change_in_development_pressure_in_conservation_areas'] = None

    # 17 - measure_direct_impact_on_conservation_areas
    # print('--direct impact on conservation areas')
    col_val['measure_direct_impact_on_conservation_areas'] = None

    # 18 - measure_change_in_impervious_area
    # print('--change in impervious area')
    col_val['measure_change_in_impervious_area'] = None

    # 19 - measure_project_use_by_residents_of_economically_disconnected_areas
    if runtype == 'link':
        eda_csv = pd.read_csv(rsp_runs_dir[rsp_name][0]+'\\Database\\rsp_evaluation\\results\\extra_links_70029.csv')
        cols = eda_csv.columns.tolist()
        col_change = dict([[col, col.strip()] for col in cols])
        eda_csv.rename(columns=col_change, inplace=True)
        eda_trips = pd.merge(eda_csv, plinks, how='left', on=['inode','jnode'])
        eda_trips = pd.merge(eda_trips, rsplink[['inode','jnode','len']], how='left', on=['inode','jnode'])
        eda_trips = eda_trips.loc[eda_trips['rsp']==1].copy()
        eda_trips['vmt'] = eda_trips['@ejvol'] * eda_trips['len']

        trips_eda = eda_trips['vmt'].sum()
        print('--project use by residents of EDAs')
        print('    ', int(trips_eda), 'vehicle miles traveled')
    else:
        trips_eda = None
    col_val['measure_project_use_by_residents_of_economically_disconnected_areas'] = trips_eda

    # 20 - measure_change_in_fine_particulate_matter_emissions_in_economically_disconnected_areas
    # print('--change in fine PM emissions in EDAs')
    pm25_edas = None
    col_val['measure_change_in_fine_particulate_matter_emissions_in_economically_disconnected_areas'] = pm25_edas

    # 21 - measure_change_in_access_to_1_barrier_jobs_for_economically_disconnected_areas
    # print('--change in EDA access to low-barrier jobs')
    jobaccess_edas = None
    col_val['measure_change_in_access_to_1_barrier_jobs_for_economically_disconnected_areas'] = jobaccess_edas

    # 22 - measure_economic_impact_due_to_industry_clustering
    # print('--econ impact due to industry clustering')
    econ_industrycluster = None
    col_val['measure_economic_impact_due_to_industry_clustering'] = econ_industrycluster

    # 23 - measure_benefit_to_key_industries
    # print('--benefit to key industries')
    key_industries = None
    col_val['measure_benefit_to_key_industries'] = key_industries

    # 24 - measure_benefits_to_areas_with_industrial_vacancy
    # print('--benefits to areas with industrial vacancy')
    vacant_industry = None
    col_val['measure_benefits_to_areas_with_industrial_vacancy'] = vacant_industry

    # 25 - measure_transit_asset_condition
    # print('--transit asset condition')
    trnt_condition = None
    col_val['measure_transit_asset_condition'] = trnt_condition

    # 26 - measure_transit_reliability
    # print('--transit reliability')
    trnt_reliability = None
    col_val['measure_transit_reliability'] = trnt_reliability

    # 27 - measure_change_in_regional_transit_ridership
    if runtype == 'line':
        orig_trnt_trips = nb_trnt['voltr'].sum()
        rsp_trnt_trips = rsp_trnt['voltr'].sum()
        change_trnt_trips = int(rsp_trnt_trips - orig_trnt_trips)
        print('--change in regional transit ridership')
        print('    ', change_trnt_trips, 'trips')
    else:
        change_trnt_trips = None
    col_val['measure_change_in_regional_transit_ridership'] = change_trnt_trips

    # 28 - measure_support_of_infill_development 
    # print('--support of infill development')
    infill_devt = None
    col_val['measure_support_of_infill_development'] = infill_devt

    # 29 - measure_change_in_congested_vehicle_hours_traveled_in_region
    if runtype == 'link':
        r_orig_cvht = nb_cvht['CongestedVHT'].sum()
        r_rsp_cvht = rsp_cvht['CongestedVHT'].sum()
        r_change_cvht = r_rsp_cvht - r_orig_cvht
        print('--change in regional congested VHT')
        print('    ', int(r_change_cvht), 'VHT')
    else:
        r_change_cvht = None
    col_val['measure_change_in_congested_vehicle_hours_traveled_in_region'] = r_change_cvht

    # 30 - measure_change_in_congested_vht_for_heavy_trucks_in_region
    if runtype == 'link':
        orig_htruck_cvht_r = nb['CongestedHTruckVHT'].sum()
        rsp_htruck_cvht_r = rsp['CongestedHTruckVHT'].sum()
        change_htruck_cvht_r = rsp_htruck_cvht_r - orig_htruck_cvht_r
        print('--change in congested vht for heavy trucks regionwide')
        print('    ', int(change_htruck_cvht_r), 'VHT')
    else:
        change_htruck_cvht_r = None
    col_val['measure_change_in_congested_vht_for_heavy_trucks_in_region'] = change_htruck_cvht_r

    # 31 - measure_bike_ped_stuff
    # print('--bike-ped stuff')
    bike_ped_stuff = None
    col_val['measure_bike_ped_stuff'] = bike_ped_stuff

    rows.append(col_val)

# final_table = pd.DataFrame(columns=rows[0].keys())
rows_df = [pd.Series(row).to_frame().T for row in rows]
final_table = pd.concat(rows_df)
final_table.to_csv(final_output)

with open(details, 'w') as file:
    file.writelines([
        'RSP Evaluation',
        f'Date: {timestamp}',
        f'Script source: {os.getcwd()}',
        f'Source: {dir}',
        f'Runs: {rsp_runs_dir.keys()}'
    ])


print(f'Calculations complete.\n Table exported to {final_output} \n Script specs exported to {details}')

Calculations for RSP20_700 (link)
--change in regional vmt
     -1805069 VMT
--change in corridor congested VHT
     -55541 VHT
--change in annual roadway fatalities and serious injuries
     -183 fatalities and serious injuries
--change in congested VHT for heavy trucks in corridor
     -4498 VHT
-- change in ghg emissions
    CO2:  -888372776.2500763 tons
    PM:   -16128.034294060082 tons
--project use by residents of EDAs
     1342337 vehicle miles traveled
--change in regional congested VHT
     -265134 VHT
--change in congested vht for heavy trucks regionwide
     -22628 VHT
Calculations for RSP57_700 (line)
-- change in ghg emissions
    CO2:  -901588896.653122 tons
    PM:   -16442.645694049308 tons
--change in regional transit ridership
     11301014 trips
Calculations complete.
 Table exported to e:\tko\TPAT\RSP_Evals\RSP_PMs_Repo\RSP_PMs\RSP_Comparison_20240510.csv 
 Script specs exported to e:\tko\TPAT\RSP_Evals\RSP_PMs_Repo\RSP_PMs\details.txt


In [ ]:
run = 'RSP20_700'

# 7 - measure_change_in_vmt
if runtype == 'link':
    print('--change in regional vmt')
    orig_vmt = nblink['AllVMT'].sum()
    rsp_vmt = rsplink['AllVMT'].sum()
    change_vmt = rsp_vmt - orig_vmt
    print('    ', int(change_vmt), 'VMT')
    
else:
    change_vmt = None
col_val['measure_change_in_vmt'] = change_vmt

# 8 - measure_change_in_congested_vht_in_corridor
if runtype == 'link':
    rsp_name='RSP20_700'

    nb_corridor = pd.read_csv(rsp_runs_dir[rsp_name][0]+'\\Database\\Select_Link\\nb_corridor_70029.csv')
    nb_corridor = nb_corridor[['INODE','JNODE']].copy()
    nb_corridor.rename(columns={'INODE':'inode','JNODE':'jnode'}, inplace=True)
    nb_corridor['corridor'] = 1
    nb = pd.merge(nblink, nb_corridor, on=['inode','jnode'], how='left')
    nb.loc[nb['corridor'].isnull(),'corridor'] = 0

    rsp_corridor = pd.read_csv(rsp_runs_dir[rsp_name][0]+'\\Database\\Select_Link\\rsp_corridor_70029.csv')
    rsp_corridor = rsp_corridor[['INODE','JNODE']].copy()
    rsp_corridor.rename(columns={'INODE':'inode','JNODE':'jnode'}, inplace=True)
    rsp_corridor['corridor'] = 1
    rsp = pd.merge(rsplink, rsp_corridor, on=['inode','jnode'], how='left')
    rsp.loc[rsp['corridor'].isnull(),'corridor'] = 0

    nb_cvht = nb.groupby('corridor').agg({'CongestedVHT':'sum'})
    rsp_cvht = rsp.groupby('corridor').agg({'CongestedVHT':'sum'})

    c_orig_cvht = nb_cvht.loc[1,'CongestedVHT'] 
    c_rsp_cvht = rsp_cvht.loc[1,'CongestedVHT']
    c_change_cvht = c_rsp_cvht - c_orig_cvht

    print('--change in corridor congested VHT')
    print('    ', int(c_change_cvht), 'VHT')
else:
    c_change_cvht = None
col_val['measure_change_in_congested_vht_in_corridor'] = c_change_cvht


NameError: name 'runtype' is not defined